In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import os
fpaths = []
labels = []
spoken = []
for f in os.listdir('Audio'):
    for w in os.listdir('Audio/' + f):
        fpaths.append('Audio/' + f + '/' + w)
        labels.append(f)
        if f not in spoken:
            spoken.append(f)
print('Words spoken:', spoken)
#spoken consists of file names

('Words spoken:', ['eye', 'apple', 'book', 'human', 'god', 'fast', 'dog', 'cat'])


In [4]:
#Files can be heard in Linux using the following commands from the command line
#cat kiwi07.wav | aplay -f S16_LE -t wav -r 8000
#Files are signed 16 bit raw, sample rate 8000
from scipy.io import wavfile

data = np.zeros((len(fpaths), 32000))
maxsize = -1
for n,file in enumerate(fpaths):
    _, d = wavfile.read(file)
    data[n, :d.shape[0]] = d
    if d.shape[0] > maxsize:
        maxsize = d.shape[0]
data = data[:, :maxsize]

#Each sample file is one row in data, and has one entry in labels
print('Number of files total:', data.shape[0])
all_labels = np.zeros(data.shape[0])
for n, l in enumerate(set(labels)):
    all_labels[np.array([i for i, _ in enumerate(labels) if _ == l])] = n
print (data[0].shape)

('Number of files total:', 120)
(3841,)


In [5]:
import python_speech_features as speech
s= speech.mfcc(data[0])
print (s.shape)

(23, 13)


In [14]:
obs =np.zeros([120,13,23])
for n,file in enumerate(fpaths):
    obs[n]=speech.mfcc(data[n]).T
print obs.shape
print obs[0]

(120, 13, 23)
[[  2.14853538e+01   2.18256874e+01   2.18838425e+01   2.16905043e+01
    2.14371424e+01   2.11805858e+01   2.10110250e+01   2.08844851e+01
    2.04754191e+01   2.03143891e+01   1.97702054e+01   1.92180935e+01
    1.88696597e+01   1.86259748e+01   1.80799053e+01  -3.60436534e+01
   -3.60436534e+01  -3.60436534e+01  -3.60436534e+01  -3.60436534e+01
   -3.60436534e+01  -3.60436534e+01  -3.60436534e+01]
 [ -1.95395294e+01  -1.90360022e+01  -1.36919105e+01  -1.19669969e+01
   -1.21882584e+01  -1.05450894e+01  -9.91025955e+00  -7.98670284e+00
   -6.10559373e+00  -8.17960020e+00  -8.25084914e+00  -6.21939461e+00
   -2.26385304e+00  -5.28682876e+00  -4.42891235e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [ -3.67843313e+01  -4.37307113e+01  -3.68920749e+01  -3.51344827e+01
   -3.43015980e+01  -3.05923617e+01  -2.66664868e+01  -2.25066896e+01
   -1.84424607e+01  -2.14096687e+01  -

In [21]:
import scipy.stats as st
import numpy as np
from __future__ import print_function

class gmmhmm:
    #This class converted with modifications from https://code.google.com/p/hmm-speech-recognition/source/browse/Word.m
    def __init__(self, n_states):
        self.n_states = n_states
        self.random_state = np.random.RandomState(0)
        
        #Normalize random initial state
        self.prior = self._normalize(self.random_state.rand(self.n_states, 1))
        self.A = self._stochasticize(self.random_state.rand(self.n_states, self.n_states))
        
        self.mu = None
        self.covs = None
        self.n_dims = None
        self.count = 0
           
    def _forward(self, B):
        log_likelihood = 0.
        T = B.shape[1]
        alpha = np.zeros(B.shape)
        for t in range(T):
            if t == 0:
                alpha[:, t] = B[:, t] * self.prior.ravel()
            else:
                alpha[:, t] = B[:, t] * np.dot(self.A.T, alpha[:, t - 1])
         
            alpha_sum = np.sum(alpha[:, t])
            #print (alpha_sum)
            alpha[:, t] /= alpha_sum
            #print (alpha[:,t])
            log_likelihood = log_likelihood + np.log(alpha_sum)
        return log_likelihood, alpha
    
    def _backward(self, B):
        T = B.shape[1]
        beta = np.zeros(B.shape);
           
        beta[:, -1] = np.ones(B.shape[0])
            
        for t in range(T - 1)[::-1]:
            beta[:, t] = np.dot(self.A, (B[:, t + 1] * beta[:, t + 1]))
            beta[:, t] /= np.sum(beta[:, t])
        return beta
    
    def _state_likelihood(self, obs):
        obs = np.atleast_2d(obs)
        B = np.zeros((self.n_states, obs.shape[1]))
        for s in range(self.n_states):
            #Needs scipy 0.14
            np.random.seed(self.random_state.randint(1))
            B[s, :] = st.multivariate_normal.pdf(
                obs.T, mean=self.mu[:, s].T, cov=self.covs[:, :, s].T)*100000000000000000000000000
            #B[s,:] /= np.sum(B[s,:])
            #print B[s,0:50]
            #This function can (and will!) return values >> 1
            #See the discussion here for the equivalent matlab function
            #https://groups.google.com/forum/#!topic/comp.soft-sys.matlab/YksWK0T74Ak
            #Key line: "Probabilities have to be less than 1,
            #Densities can be anything, even infinite (at individual points)."
            #This is evaluating the density at individual points...
        
        #print(B.shape)
        #print (B[0,:])
        #self.count += 1
        #print("ek")
        return B
    
    def _normalize(self, x):
        return (x + (x == 0)) / np.sum(x)
    
    def _stochasticize(self, x):
        return (x + (x == 0)) / np.sum(x, axis=1)
    
    def _em_init(self, obs):
        #Using this _em_init function allows for less required constructor args
        if self.n_dims is None:
            self.n_dims = obs.shape[0]
        if self.mu is None:
            #??
            subset = self.random_state.choice(np.arange(self.n_dims), size=self.n_states, replace=False)
            self.mu = obs[:, subset]
        if self.covs is None:
            self.covs = np.zeros((self.n_dims , self.n_dims,self.n_states))
            #print('obs', obs.shape)
            self.covs += np.diag(np.diag(np.cov(obs)))[:, :, None]
            #print(self.covs.shape)
        return self
    
    def _em_step(self, obs): 
        obs = np.atleast_2d(obs)
        B = self._state_likelihood(obs)
        T = obs.shape[1]
        
        log_likelihood, alpha = self._forward(B)
        beta = self._backward(B)
        
        xi_sum = np.zeros((self.n_states, self.n_states)) #??
        gamma = np.zeros((self.n_states, T))
        
        for t in range(T - 1):
            partial_sum = self.A * np.dot(alpha[:, t], (beta[:, t] * B[:, t + 1]).T)
            xi_sum += self._normalize(partial_sum)
            partial_g = alpha[:, t] * beta[:, t]
            gamma[:, t] = self._normalize(partial_g)
              
        partial_g = alpha[:, -1] * beta[:, -1]
        gamma[:, -1] = self._normalize(partial_g)
        
        expected_prior = gamma[:, 0]
        expected_A = self._stochasticize(xi_sum)
        
        expected_mu = np.zeros((self.n_dims, self.n_states))
        expected_covs = np.zeros((self.n_dims, self.n_dims, self.n_states))
        
        gamma_state_sum = np.sum(gamma, axis=1)
        #Set zeros to 1 before dividing
        gamma_state_sum = gamma_state_sum + (gamma_state_sum == 0)
        
        for s in range(self.n_states):
            gamma_obs = obs * gamma[s, :] #??
            expected_mu[:, s] = np.sum(gamma_obs, axis=1) / gamma_state_sum[s]
            partial_covs = np.dot(gamma_obs, obs.T) / gamma_state_sum[s] - np.dot(expected_mu[:, s], 
                                                                                  expected_mu[:, s].T)
            #Symmetrize
            partial_covs = np.triu(partial_covs) + np.triu(partial_covs).T - np.diag(partial_covs) #??
        
        #Ensure positive semidefinite by adding diagonal loading
        expected_covs += .01 * np.eye(self.n_dims)[:, :, None] #??
        
        self.prior = expected_prior
        #print self.prior
        self.mu = expected_mu
    
        #print self.mu
        self.covs = expected_covs
        self.A = expected_A
        print (B)
        return log_likelihood
    
    def fit(self, obs, n_iter=1):
        #Support for 2D and 3D arrays
        #2D should be n_features, n_dims
        #3D should be n_examples, n_features, n_dims
        #For example, with 6 features per speech segment, 105 different words
        #this array should be size
        #(105, 6, X) where X is the number of frames with features extracted
        #For a single example file, the array should be size (6, X)
        if len(obs.shape) == 2:
            for i in range(n_iter):
                self._em_init(obs)
                log_likelihood = self._em_step(obs)
        elif len(obs.shape) == 3:
            count = obs.shape[0]
            for n in range(count):
                for i in range(n_iter):
                    self._em_init(obs[n, :, :])
                    log_likelihood = self._em_step(obs[n, :, :])
        return self
    
    def transform(self, obs):
        #Support for 2D and 3D arrays
        #2D should be n_features, n_dims
        #3D should be n_examples, n_features, n_dims
        #For example, with 6 features per speech segment, 105 different words
        #this array should be size
        #(105, 6, X) where X is the number of frames with features extracted
        #For a single example file, the array should be size (6, X)
        if len(obs.shape) == 2:
            B = self._state_likelihood(obs)
            log_likelihood, _ = self._forward(B)
            return log_likelihood
        elif len(obs.shape) == 3:
            count = obs.shape[0]
            out = np.zeros((count,))
            for n in range(count):
                B = self._state_likelihood(obs[n, :, :])
                log_likelihood, _ = self._forward(B)
                out[n] = log_likelihood
            return out

if __name__ == "__main__":
    rstate = np.random.RandomState(0)
    t1 = np.ones((4, 40)) + .001 * rstate.rand(4, 40)
    t1 /= t1.sum(axis=0)
    t2 = rstate.rand(*t1.shape)
    t2 /= t2.sum(axis=0)
    
    m1 = gmmhmm(2)
    m1.fit(t1)
    m2 = gmmhmm(2)
    m2.fit(t2)
    
    m1t1 = m1.transform(t1)
    m2t1 = m2.transform(t1)
    print("Likelihoods for test set 1")
    print("M1:", m1t1)
    print("M2:", m2t1)
    print("Prediction for test set 1")
    print("Model", np.argmax([m1t1, m2t1]) + 1)
    print()
    
    m1t2 = m1.transform(t2)
    m2t2 = m2.transform(t2)
    print("Likelihoods for test set 2")
    print("M1:", m1t2)
    print("M2:", m2t2)
    print("Prediction for test set 2")
    print("Model", np.argmax([m1t2, m2t2]) + 1)

[[  7.50186208e+40   1.68236036e+41   5.48582952e+39   2.29029672e+40
    4.47549061e+40   8.65853237e+40   3.32172293e+40   4.25114714e+40
    9.96290375e+40   3.93130977e+39   1.27505742e+41   6.67215723e+40
    4.91797505e+39   1.78487477e+40   1.47413481e+40   1.63822274e+40
    1.60145010e+39   2.84975243e+40   7.90442178e+40   3.87364358e+40
    2.75719740e+40   8.94060853e+40   2.88920435e+40   7.82292316e+39
    4.10517237e+39   9.80877384e+40   1.33109761e+39   1.91575129e+40
    2.24865124e+40   5.57341586e+38   2.27147759e+39   6.00914906e+40
    2.68908123e+39   9.24472244e+40   1.49153783e+38   1.95406223e+40
    1.78363789e+40   6.37299678e+40   2.27912546e+40   1.24464155e+40]
 [  2.69373256e+40   2.29029672e+40   1.86912629e+38   1.68236036e+41
    8.89274132e+39   3.15425371e+39   1.41476084e+40   1.68164272e+39
    3.78263452e+40   1.48958419e+40   6.35412428e+39   4.56907632e+40
    7.90641218e+36   1.55461636e+40   2.56473090e+40   2.10831020e+40
    1.87510855e+39 

In [22]:
#print (count)

In [23]:
from sklearn.cross_validation import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(all_labels, test_size=0.1, random_state=0)

#for n,i in enumerate(obs):
#    obs[n] /= obs[n].sum(axis=0)

for train_index, test_index in sss:
    X_train, X_test = obs[train_index, ...], obs[test_index, ...]
    y_train, y_test = all_labels[train_index], all_labels[test_index]
print('Size of training matrix:', X_train.shape)
print('Size of testing matrix:', X_test.shape)

Size of training matrix: (112, 13, 23)
Size of testing matrix: (8, 13, 23)


In [24]:
ys = set(all_labels)
ms = [gmmhmm(6) for y in ys]
_ = [m.fit(X_train[y_train == y, :, :]) for m, y in zip(ms, ys)]
ps = [m.transform(X_test) for m in ms]
res = np.vstack(ps)
predicted_labels = np.argmax(res, axis=0)
missed = (predicted_labels != y_test)
print('Test accuracy: %.2f percent' % (100 * (1 - np.mean(missed))))

[[  1.94123732e+05   2.11361353e+05   2.19007226e+05   3.83280812e+06
    3.60786539e+07   1.85150790e+06   1.52208687e+04   5.31758904e+01
    1.64125189e+03   1.87382099e+04   9.16940681e+03   7.11746209e+03
    1.99179069e+03   1.67232642e+03   9.65470215e+01   6.24475858e-03
    4.19669092e-02   4.19669092e-02   4.19669092e-02   4.19669092e-02
    4.19669092e-02   4.19669092e-02   4.19669092e-02]
 [  2.23066178e+02   4.34119481e+00   1.81967512e+00   2.80884471e+01
    1.99179069e+03   5.34022045e+03   2.23930869e+02   4.56967556e+02
    1.32044573e+05   1.21242292e+05   1.52520399e+06   1.41302279e+07
    3.60786539e+07   1.10008500e+07   6.55949922e+05   7.00890641e+02
    6.79362904e+00   6.79362904e+00   6.79362904e+00   6.79362904e+00
    6.79362904e+00   6.79362904e+00   6.79362904e+00]
 [  1.30421880e+03   5.81863476e+03   1.69385127e+03   1.29476653e+04
    1.85150790e+06   3.60786539e+07   6.03923082e+06   6.55104734e+04
    1.39200640e+05   2.34364651e+04   2.20850128e+04

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:32: RuntimeWarning: invalid value encountered in divide
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:34: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:45: RuntimeWarning: invalid value encountered in divide


[[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
   nan  nan  nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
   nan  nan  nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
   nan  nan  nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
   nan  nan  nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
   nan  nan  nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
   nan  nan  nan  nan  nan  nan  nan  nan  nan]]
[[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
   nan  nan  nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
   nan  nan  nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan  n

In [19]:
print count

SyntaxError: invalid syntax (<ipython-input-19-07bd36c4cd43>, line 1)